# PYNQ로 MMIO 사용하기

## 목표

이 notebook의 목적은 MMIO (메모리 매핑 된 I / O) PYNQ 클래스를 사용하는 방법을 보여주는 것이다.   

## 하드웨어 디자인

이 예제는 LED, 버튼 및 스위치에 연결된 3 개의 AXI GPIO 컨트롤러가 있는 이전 예제와 동일한 비트 스트림을 사용한다. AXI GPIO LED 스위치를 읽고 쓸 수있는 PYNQ 드라이버가 있지만 데모 용으로 AXI GPIO 컨트롤러를 사용하여 PYNQ MMIO 클래스를 사용하는 방법을 보여준다.

![AXI GPIO Design](./images/axi_gpio_design.png "AXI GPIO Design")

이 notebook은 이전 실습과 매우 유사해 보일 것이다. 유사한 방법으로 버튼, 스위치 및 LED를 연습 할 예정이지만 MMIO 클래스를 직접 사용하고 있으며 코드에 약간의 차이가 있음에 유의해야 한다. MMIO 클래스의 경우, 우리는 읽거나 쓸 오프셋 주소를 지정해야 한다.

*LED*, *switches* 및 *buttons* 클래스의 드라이버 코드를 검사하면 PYNQ MMIO 클래스를 사용하는 것을 알 수 있다.

### 1. axi_gpio.bit overlay를 다운로드 한다.

In [ ]:
from pynq import Overlay
axi_gpio_design = Overlay("./bitstream/axi_gpio.bit")

이 디자인에 대한 IP Dictionary를 확인한다.


실제 메모리 주소를 기록하라. 이것은 MMIO 클래스와 함께 사용되어 읽고 쓸 메모리 영역에 매핑한다.

In [ ]:
axi_gpio_design.ip_dict

## MMIO 클래스

MMIO는 배열 또는 주소 범위를 매핑 할 수 있다. 물리적 메모리 주소와 범위는 MMIO 클래스에서 필요하다.

이 예제에서 MMIO 클래스는 AXI GPIO의 레지스터 공간에 직접 액세스하고 IP를 제어하는 데 사용된다.


AXI GPIO 컨트롤러에는 2 개의 채널이 있다. 설계시 각 AXI 컨트롤러의 채널이 하나만 사용된다 (이전 실험에서 설명한대로).

우리는 두 개의 레지스터만 사용한다:

데이터 레지스터는 오프셋인 0x0에 매핑되고, 삼중 레지스터는 오프셋 0x4에 매핑된다. AXI GPIO를 사용하려면 IP를 입력 또는 출력으로 설정하기 위해 3 상태 레지스터* (tri-state register) *를 설정해야 한다. 데이터 레지스터는 읽거나 쓸 수 있다. 예를 들면, LED에 연결된 AXI GPIO는 IP를 출력으로 구성하기 위해 3 상태 레지스터를 설정한다. LED는 데이터 레지스터에 기록 된 값에 따라 켜지거나 꺼진다. 버튼 또는 스위치의 경우 IP는 __입력__으로 구성되며 데이터 레지스터의 값은 스위치 또는 버튼의 위치에 해당하는 값이 된다.

다음 예제에서 각 AXI GPIO에 해당하는 3 개의 MMIO 인스턴스가 생성된다.

먼저 컨트롤러의 물리적 주소를 파이썬 변수에 할당한다.

In [ ]:
buttons_address = axi_gpio_design.ip_dict['buttons']['phys_addr']
switches_address = axi_gpio_design.ip_dict['switches']['phys_addr']
leds_address = axi_gpio_design.ip_dict['leds']['phys_addr']

print("Physical address of button: 0x" + format(buttons_address, '02x'))
print("Physical address of button: 0x" + format(switches_address, '02x'))
print("Physical address of button: 0x" + format(leds_address, '02x'))

### 1. 스위치 및 푸시 버튼을 다시 제어하라

`MMIO` 인스턴스는 주소와 선택적으로 범위로 생성된다. 범위는 기본 주소에서 액세스 할 수있는 주소 범위를 지정한다. 물리적으로 존재하는 시스템에서 주소를 읽고 쓸 때 주의를 기울여야 한다. 액세스 할 수없는 위치를 읽거나 쓰면 시스템이 중지 될 수 있다.

In [ ]:
from pynq import MMIO
RANGE = 8 # Number of bytes; 8/4 = 2x 32-bit locations which is all we need for this example
buttons = MMIO(buttons_address, RANGE) 

IO를 입력으로 구성하려면 0xffffffff를 오프셋이 0x4인 3 상태 레지스터에 쓴다.

In [ ]:
buttons.write(0x4, 0xffffffff) 

In [ ]:
print(f"Push-buttons: {buttons.read()}")

이전과 마찬가지로 위의 셀을 다시 실행하는 동안 푸시 버튼 조합을 눌러보시오.

스위치를위한 AXI GPIO 컨트롤러도 비슷한 방식으로 사용할 수 있다 :

In [ ]:
switches = MMIO(switches_address, RANGE)
switches.write(0x4, 0xffffffff) 

In [ ]:
print(f"Switches: {switches.read()}")

### 2. LED를 다시 조작하라

LED는 유사한 방식으로 사용될 수 있으며, 이번에는 '0x0'이 3 상태 레지스터에 쓰여 IO를 출력으로 구성한다.

In [ ]:
leds = MMIO(leds_address, RANGE)
leds.write(0x4, 0x0) # Write 0x0 to location 0x4; Set tri-state to output

In [ ]:
leds.write(0x0, 0xf) # Write 0xf to location 0x0 (Turn on the LEDs)

### 3. 모두Putting it together (again!)

이전과 마찬가지로 LED를 푸시 버튼의 값으로 설정하는 루프를 실행한다.


다음 셀을 실행하기 전에 스위치 0 (SW0)이 "켜짐"인지 확인하시오. 루프가 작동하는 동안 푸시 버튼 단추를 누르고 해당 LED가 켜지는지 확인하시오. 루프가 있으려면 Switch 0을 off로 변경하시오.

In [ ]:
while((switches.read(0x0)&1) is 1):
    leds.write(0x0, buttons.read(0x0))